# Data Loading, Storage, and File Formats

***

Author: Jiacheng

Time: 2019-05-13

***

In [1]:
import pandas as pd
import numpy as np

## 1. Reading and Writing Data in Text Format
## 读写文本格式的数据

In [2]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,fini


In [4]:
pd.read_table('examples/ex1.csv', sep=',')

/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,fini


In [5]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,go


In [6]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'mess'])

,a,b,c,d,mess
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,go


In [8]:
names=['a', 'b', 'c', 'd', 'mess']
pd.read_csv('examples/ex2.csv', names=names, index_col='mess')

,a,b,c,d
mess,,,,
hello,1,2,3,4
world,5,6,7,8
go,9,10,11,12


In [9]:
# 多层次化索引
parsed = pd.read_csv('examples/csv_multi.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [10]:
 list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100493\n']

#### read_table is deprecated, so we use read_csv

In [16]:
result = pd.read_csv('examples/ex3.txt', '\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100493


In [17]:
pd.read_csv('examples/ex4.csv')

,,,,# hey!
a,b,c,d,message
# just wanted to make things more difficult for you,NaN,NaN,NaN,NaN
# who reads CSV files with computers,anyway?,NaN,NaN,NaN
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,go


In [18]:
pd.read_csv('examples/ex4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,go


In [19]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,go


In [20]:
result = pd.read_csv('examples/ex5.csv', na_values=['N'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,go


In [22]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
result = pd.read_csv('examples/ex5.csv', na_values=sentinels)
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,go


### Reading Text Files in Pieces
### 逐块读取文本文件

In [23]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,fini


In [25]:
result = pd.read_csv('examples/ex6.csv')
result

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo
3,a,b,c,d,message
4,1,2,3,4,hello
5,5,6,7,8,world
6,9,10,11,12,go
7,a,b,c,d,message
8,1,2,3,4,hello
9,5,6,7,8,world


In [26]:
pd.read_csv('examples/ex6.csv', nrows=5)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo
3,a,b,c,d,message
4,1,2,3,4,hello


In [31]:
# 指针
chunker = pd.read_csv('examples/ex6.csv', chunksize=10) # 每10个进行一次取出
chunker

In [30]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=10)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['message'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot

world      6.0
hello      6.0
message    5.0
foo        4.0
go         2.0
dtype: float64

In [32]:
result['message'].value_counts()

hello      6
world      6
message    5
foo        4
go         2
Name: message, dtype: int64

In [34]:
# get_chunk
chunker = pd.read_csv('examples/ex6.csv', iterator=True)
chunker.get_chunk(4)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo
3,a,b,c,d,message


### Writing Data to Text Format
### 将数据写出到文本格式

In [36]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,go


In [37]:
data.to_csv('examples/out.csv')

In [38]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|go


In [39]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,go


In [40]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,go


In [42]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [44]:
dates = pd.date_range('10/5/2019', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/tseries.csv')

/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


### Working with Delimited Formats
### 处理分隔符格式

In [2]:
import csv

In [3]:
f = open('examples/ex7.csv')
reader = csv.reader(f)

In [4]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [6]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
header, values = lines[0], lines[1:]

data_dict = {h:v for h, v in zip(header, zip(*values))}

print(data_dict)

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}


In [15]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [16]:
with open('examples/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

### JSON Data
### JSON数据

In [17]:
import json

In [18]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [19]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [20]:
asjson = json.dumps(result)

In [23]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age', 'pets'])
siblings

,name,age,pets
0,Scott,30,"[Zeus, Zuko]"
1,Katie,38,"[Sixes, Stache, Cisco]"


In [24]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [25]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [26]:
 print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML: Web Scraping
### XML和HTML：Web信息收集

In [18]:
import lxml

tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [19]:
len(tables)

1

In [20]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [21]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

### Parsing XML with lxml.objectify
### 利用lxml.objectify解析XML

In [26]:
from lxml import objectify

path = 'examples/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [27]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [28]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [29]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [30]:
root

<Element a at 0x124ccd108>

In [31]:
root.get('href')

'http://www.google.com'

In [33]:
root.text

'Google'

--- 

## 2. Binary Data Formats
## 二进制数据格式

In [2]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,fini


In [3]:
# 二进制格式存储
frame.to_pickle('examples/frame_pickle')

In [4]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,fini


### Using HDF5 Format
### 使用HDF5格式

In [11]:
frame = pd.DataFrame({'a': np.random.randn(10)})
frame

,a
0,-0.033298
1,0.474904
2,-0.817799
3,-2.601132
4,0.865542
5,-0.476407
6,-0.824612
7,-0.086109
8,0.057744
9,-0.061907


In [12]:
store = pd.HDFStore('examples/mydata.h5')

In [13]:
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: examples/mydata.h5

In [14]:
store['obj1']

,a
0,-0.033298
1,0.474904
2,-0.817799
3,-2.601132
4,0.865542
5,-0.476407
6,-0.824612
7,-0.086109
8,0.057744
9,-0.061907


In [15]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 3 and index <= 7'])

,a
3,-2.601132
4,0.865542
5,-0.476407
6,-0.824612
7,-0.086109


In [16]:
store.close()

In [17]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.033298
1,0.474904
2,-0.817799
3,-2.601132
4,0.865542


### Reading Microsoft Excel Files
### 读取Microsoft Excel文件

In [22]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [24]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()
# frame.to_excel('examples/ex2.xlsx')

---

## 3. Interacting with Web APIs
## Web APIs交互

In [25]:
import requests

In [26]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [27]:
data = resp.json()
data[0]['title']

'CLN: Cleanup code in parsers.pyx'

In [29]:
issues = pd.DataFrame(data, columns=['number', 'title','url', 'state'])
issues


,number,title,url,state
0,26386,CLN: Cleanup code in parsers.pyx,https://api.github.com/repos/pandas-dev/pandas...,open
1,26385,GroupBy nth ignores observed keyword for Categ...,https://api.github.com/repos/pandas-dev/pandas...,open
2,26384,Remove Deprecated Azure CI Step,https://api.github.com/repos/pandas-dev/pandas...,open
3,26383,Prefixed batch script commands with call,https://api.github.com/repos/pandas-dev/pandas...,open
4,26381,BUG: error calculating BusinessHourMixin.apply...,https://api.github.com/repos/pandas-dev/pandas...,open
5,26380,ENH: Implementing NEP 18's __array_function__,https://api.github.com/repos/pandas-dev/pandas...,open
6,26379,BUG: Series.combine_first with mixed timezones,https://api.github.com/repos/pandas-dev/pandas...,open
7,26377,Fix type annotations in pandas.core.ops,https://api.github.com/repos/pandas-dev/pandas...,open
8,26376,pd.read_fwf fails with file pointer to url,https://api.github.com/repos/pandas-dev/pandas...,open
9,26374,BUG: ngroups and len(groups) do not equal when...,https://api.github.com/repos/pandas-dev/pandas...,open


---

## 4. Interacting with Databases
## 数据库交互

In [63]:
import sqlite3

In [66]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);
"""

con = sqlite3.connect('examples/mydata.sqlite')

In [67]:
con.execute(query)
con.commit()

In [68]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)

In [69]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [70]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [71]:
cursor.rowcount

-1

In [72]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [73]:
import sqlalchemy as sqla

In [74]:
db = sqla.create_engine('sqlite:///examples/mydata.sqlite')

In [75]:
pd.read_sql('select * from test', db)

,a,b,c,d
